# Exploring Spark with Pandas


Using pandas examples, convert the analysis to pyspark. This is useful if you discover your data grows too large for your tooling.

The purpose of this notebook is to familiarise yourself you the pyspark API. You are welcome to use the R version of this if you wish. As long as you are able to obtain the correct results. We will be using python in this notebook as it is quite widely used through data science and the community is very large.



#### Firstly, let's get our spark session

In [3]:
!pip install pyspark -qq

     |████████████████████████████████| 281.3 MB 45 kB/s 
     |████████████████████████████████| 199 kB 48.1 MB/s 


In [4]:
from pyspark.sql import SparkSession
import pandas as pd 
spark = SparkSession.builder.appName('panda-and-spark').getOrCreate()

### Overview


* Joining two dataframes/data sets
* Simple aggregations
* Persisting

#### JOIN: Pandas

We won't use this more in this notebook, but observe how the joins work.

We what happens if you change from the default inner join to outer joins.

In [5]:
customer_raw = [(1, 'bob', 3462543658686),
           (2, 'rob', 9087567565439),
           (3, 'tim', 5436586999467),
           (4, 'tom', 8349756853250)]

customer_cols = ['id', 'name', 'credit_card_number']



orders_raw = [(1, 'ketchup', 'bob', 1.20),
           (2, 'rutabaga', 'bob', 3.35),
           (3, 'fake vegan meat', 'rob', 13.99),
           (4, 'cheesey poofs', 'tim', 3.99),
           (5, 'ice cream', 'tim', 4.95),
           (6, 'protein powder', 'tom', 49.95)]

orders_cols = ['id', 'product_name', 'customer', 'price']

In [6]:
customer_df = pd.DataFrame(customer_raw, columns=customer_cols)
orders_df = pd.DataFrame(orders_raw, columns=orders_cols)

customer_df

joined_df = pd.merge(customer_df, orders_df, how='inner', left_on='name', right_on='customer')
joined_df

## For self study. What happens if (4, 'tom', 8349756853250) in valuesA becomes (4, 'tod', 8349756853250)
## How do the results change?
## More sensibly; what if customers have not made any orders but we still require them in the result set? 

,id_x,name,credit_card_number,id_y,product_name,customer,price
0,1,bob,3462543658686,1,ketchup,bob,1.20
1,1,bob,3462543658686,2,rutabaga,bob,3.35
2,2,rob,9087567565439,3,fake vegan meat,rob,13.99
3,3,tim,5436586999467,4,cheesey poofs,tim,3.99
4,3,tim,5436586999467,5,ice cream,tim,4.95
5,4,tom,8349756853250,6,protein powder,tom,49.95


#### JOIN: Spark

In [7]:
customersDF = spark.createDataFrame(customer_raw, customer_cols)

ordersDF = spark.createDataFrame(orders_raw, orders_cols)

# Show tables
customersDF.show()
ordersDF.show()

+---+----+------------------+
| id|name|credit_card_number|
+---+----+------------------+
|  1| bob|     3462543658686|
|  2| rob|     9087567565439|
|  3| tim|     5436586999467|
|  4| tom|     8349756853250|
+---+----+------------------+

+---+---------------+--------+-----+
| id|   product_name|customer|price|
+---+---------------+--------+-----+
|  1|        ketchup|     bob|  1.2|
|  2|       rutabaga|     bob| 3.35|
|  3|fake vegan meat|     rob|13.99|
|  4|  cheesey poofs|     tim| 3.99|
|  5|      ice cream|     tim| 4.95|
|  6| protein powder|     tom|49.95|
+---+---------------+--------+-----+



In [8]:
joinedDF = customersDF.join(ordersDF, customersDF.name == ordersDF.customer)
joinedDF.show()

+---+----+------------------+---+---------------+--------+-----+
| id|name|credit_card_number| id|   product_name|customer|price|
+---+----+------------------+---+---------------+--------+-----+
|  1| bob|     3462543658686|  1|        ketchup|     bob|  1.2|
|  1| bob|     3462543658686|  2|       rutabaga|     bob| 3.35|
|  2| rob|     9087567565439|  3|fake vegan meat|     rob|13.99|
|  3| tim|     5436586999467|  4|  cheesey poofs|     tim| 3.99|
|  3| tim|     5436586999467|  5|      ice cream|     tim| 4.95|
|  4| tom|     8349756853250|  6| protein powder|     tom|49.95|
+---+----+------------------+---+---------------+--------+-----+



## Simple Aggregations

Now let's explore simple aggregations. You will be using these often when doing exploratory work in big data. Remember, the intention here is that you grow familiar with the way the API works, and how to translate inquiries into that API.

> _How much did each person spend?_

In [9]:
joined_df.groupby('name').agg({"price": ["sum"]}) 

,price
,sum
name,
bob,4.55
rob,13.99
tim,8.94
tom,49.95


In [10]:
import pyspark.sql.functions as f

joinedDF.groupby('name').agg(f.sum('price').alias('total')).show()

+----+-----------------+
|name|            total|
+----+-----------------+
| bob|             4.55|
| rob|            13.99|
| tim|8.940000000000001|
| tom|            49.95|
+----+-----------------+



Let's use bigger data
  * NYC crash data

In [11]:
# save to the filesystem to prevent another load
! curl -o rows.csv https://data.cityofnewyork.us/api/views/h9gi-nx95/rows.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  391M    0  391M    0     0  3833k      0 --:--:--  0:01:44 --:--:-- 4389k


In [12]:
import pandas as pd
nyc_df = pd.read_csv('rows.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [13]:
# number or rows

print(len(nyc_df))

# this is quite large so we will work with a sample while we experiment in pandas as least.

1917113


We'll take a random sample at 20% of the original data

In [14]:
nyc_small = nyc_df.sample(frac=0.2, replace=False, random_state=1)

In [15]:
# we are also going to limit the columns to those we are going to work with

nyc_small = nyc_small[['CRASH DATE', 'CONTRIBUTING FACTOR VEHICLE 1', 
                       'BOROUGH', 'VEHICLE TYPE CODE 1', 
                       'NUMBER OF PERSONS INJURED']]

In [20]:
nyc_small.head()

,CRASH DATE,CONTRIBUTING FACTOR VEHICLE 1,BOROUGH,VEHICLE TYPE CODE 1,NUMBER OF PERSONS INJURED
508626,12/08/2018,Backing Unsafely,BROOKLYN,Sedan,0.0
453827,03/06/2019,Failure to Yield Right-of-Way,QUEENS,Sedan,0.0
881868,05/15/2017,Passing or Lane Usage Improper,BROOKLYN,Sedan,0.0
450342,03/29/2019,Passing Too Closely,NaN,Sedan,0.0
260533,11/23/2019,Unspecified,QUEENS,Sedan,0.0


Now, let's create the pyspark dataframe. Now we two frames with the same content
  * nyc_small: pandas
  * sdf_small: pyspark

In [17]:
from pyspark.sql import SQLContext


# there are nan's in the frame with strings, and spark can't 'infer the schema', so we have to help it out 
# by replacing them with empty strings and forcing the column to be a string

sdf_small = SQLContext(spark).createDataFrame(nyc_small.fillna('').astype('str'))


# Lets check the schema quickly

print(sdf_small.schema)

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:114: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning,


StructType([StructField('CRASH DATE', StringType(), True), StructField('CONTRIBUTING FACTOR VEHICLE 1', StringType(), True), StructField('BOROUGH', StringType(), True), StructField('VEHICLE TYPE CODE 1', StringType(), True), StructField('NUMBER OF PERSONS INJURED', StringType(), True)])


In [37]:
#create a temporary table to query to 

sdf_small.registerTempTable('data')

/usr/local/lib/python3.7/dist-packages/pyspark/sql/dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


# Questions

Answer the following questions by porting the pandas code to the Spark API



# Question 1


> On what day do most crashes occcur?

In [18]:
# Pandas
nyc_small.groupby('CRASH DATE')['CRASH DATE'].count().sort_values(ascending=False).head(5)

CRASH DATE
01/21/2014    230
11/15/2018    225
05/19/2017    218
01/18/2015    214
05/18/2017    198
Name: CRASH DATE, dtype: int64

In [52]:
### Spark?

select = spark.sql("""
      SELECT `CRASH DATE`, count(*) as freq
      FROM data
      GROUP BY `CRASH DATE`
      ORDER BY 2 DESC
      LIMIT 5
      
                           """)
select.show()

+----------+----+
|CRASH DATE|freq|
+----------+----+
|01/21/2014| 230|
|11/15/2018| 225|
|05/19/2017| 218|
|01/18/2015| 214|
|05/18/2017| 198|
+----------+----+



# Question 2

> _Where do most crashes occur?_

In [21]:
nyc_small.groupby('BOROUGH')['BOROUGH'].count().sort_values(ascending=False).head(5)

BOROUGH
BROOKLYN         83574
QUEENS           70756
MANHATTAN        60411
BRONX            38624
STATEN ISLAND    11084
Name: BOROUGH, dtype: int64

In [62]:
## Spark?

select = spark.sql("""
      SELECT BOROUGH, count(*) as freq
      FROM data
      GROUP BY BOROUGH
      ORDER BY 2 DESC
      LIMIT 6
      
                           """)
select.show()


+-------------+------+
|      BOROUGH|  freq|
+-------------+------+
|             |118974|
|     BROOKLYN| 83574|
|       QUEENS| 70756|
|    MANHATTAN| 60411|
|        BRONX| 38624|
|STATEN ISLAND| 11084|
+-------------+------+



 # Question 3
 
 > What is the most common cause of accident in 'QUEENS'

In [64]:
nyc_small.head()

,CRASH DATE,CONTRIBUTING FACTOR VEHICLE 1,BOROUGH,VEHICLE TYPE CODE 1,NUMBER OF PERSONS INJURED
508626,12/08/2018,Backing Unsafely,BROOKLYN,Sedan,0.0
453827,03/06/2019,Failure to Yield Right-of-Way,QUEENS,Sedan,0.0
881868,05/15/2017,Passing or Lane Usage Improper,BROOKLYN,Sedan,0.0
450342,03/29/2019,Passing Too Closely,NaN,Sedan,0.0
260533,11/23/2019,Unspecified,QUEENS,Sedan,0.0


In [22]:
nyc_small[(nyc_small.BOROUGH == 'QUEENS')]['CONTRIBUTING FACTOR VEHICLE 1'].value_counts()

# you can also use a group by (to avoid the pandas value_counts function)

nyc_small[(nyc_small.BOROUGH == 'QUEENS')].groupby(
    'CONTRIBUTING FACTOR VEHICLE 1'
)['CONTRIBUTING FACTOR VEHICLE 1'].count().sort_values(ascending=False).head(5)

CONTRIBUTING FACTOR VEHICLE 1
Unspecified                       25525
Driver Inattention/Distraction    14945
Failure to Yield Right-of-Way      6029
Backing Unsafely                   3461
Following Too Closely              2468
Name: CONTRIBUTING FACTOR VEHICLE 1, dtype: int64

In [130]:
## Spark?

select = spark.sql("""
      SELECT `CONTRIBUTING FACTOR VEHICLE 1`, count(*) as freq
      FROM data
      WHERE BOROUGH = 'QUEENS'
      GROUP BY `CONTRIBUTING FACTOR VEHICLE 1`
      ORDER BY 2 DESC
      LIMIT 5
      
                           """)
select.show()

+-----------------------------+-----+
|CONTRIBUTING FACTOR VEHICLE 1| freq|
+-----------------------------+-----+
|                  Unspecified|25525|
|         Driver Inattentio...|14945|
|         Failure to Yield ...| 6029|
|             Backing Unsafely| 3461|
|         Following Too Clo...| 2468|
+-----------------------------+-----+



# Question 4

> _What is the average number or injuries for specific cars driving in specific suburbs_


In [72]:
nyc_small.head()

,CRASH DATE,CONTRIBUTING FACTOR VEHICLE 1,BOROUGH,VEHICLE TYPE CODE 1,NUMBER OF PERSONS INJURED
508626,12/08/2018,Backing Unsafely,BROOKLYN,Sedan,0.0
453827,03/06/2019,Failure to Yield Right-of-Way,QUEENS,Sedan,0.0
881868,05/15/2017,Passing or Lane Usage Improper,BROOKLYN,Sedan,0.0
450342,03/29/2019,Passing Too Closely,NaN,Sedan,0.0
260533,11/23/2019,Unspecified,QUEENS,Sedan,0.0


In [23]:
nyc_small.groupby(['VEHICLE TYPE CODE 1', 'BOROUGH'])['NUMBER OF PERSONS INJURED'].mean().sort_values(ascending=False).head(3)

VEHICLE TYPE CODE 1  BOROUGH  
FRONT                BROOKLYN     12.0
MTA B                MANHATTAN     9.0
TRUCK VAN            BRONX         7.0
Name: NUMBER OF PERSONS INJURED, dtype: float64

In [157]:
## Spark?


select = spark.sql("""
      SELECT `VEHICLE TYPE CODE 1`, BOROUGH, avg(`NUMBER OF PERSONS INJURED`) as average
      FROM data
      group by `VEHICLE TYPE CODE 1`, BOROUGH
      order by 3 DESC
      LIMIT 3
      
                           """)
select.show()


+-------------------+---------+-------+
|VEHICLE TYPE CODE 1|  BOROUGH|average|
+-------------------+---------+-------+
|              FRONT| BROOKLYN|   12.0|
|              MTA B|MANHATTAN|    9.0|
|          TRUCK VAN|    BRONX|    7.0|
+-------------------+---------+-------+

